In [7]:
import pandas as pd
import time
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ========== STEP 1: Load input links ==========

input_file = "final_products.csv"
df_links = pd.read_csv(input_file)

if "Product Link" not in df_links.columns:
    raise ValueError("CSV must contain a column named 'Product Link'.")

product_links = df_links["Product Link"].dropna().unique().tolist()
print(f"🔗 Total product links found: {len(product_links)}")

# ========== Helper: Initialize WebDriver ==========
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")  # remove if you want to see browser
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    driver.set_page_load_timeout(30)
    return driver

driver = create_driver()

# ========== STEP 2: Extraction Loop ==========
all_data = []
output_file = "lenskart_product_technical_info.csv"

for idx, url in enumerate(product_links, start=1):
    print(f"\n[{idx}/{len(product_links)}] 🔍 Extracting: {url}")
    product_data = {"Product Link": url}  # primary key

    try:
        driver.get(url)
        time.sleep(4)

        # Scroll to trigger data load
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
        time.sleep(2)

        # Click 'Show All Information'
        show_info_button = WebDriverWait(driver, 8).until(
            EC.presence_of_element_located((By.XPATH, "//div[@role='button' and contains(., 'Show All Information')]"))
        )
        driver.execute_script("arguments[0].click();", show_info_button)

        # Wait for modal to appear
        WebDriverWait(driver, 8).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'modal-body')]"))
        )
        time.sleep(1)

        # Extract key-value data
        rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'modal-body')]//div[contains(@class, 'fkAzwm')]")
        for row in rows:
            try:
                key = row.find_element(By.CLASS_NAME, "jCqYEQ").text.strip()
                value = row.find_element(By.CLASS_NAME, "hTEVyJ").text.strip()
                product_data[key] = value
            except:
                continue

        print(f"✅ Extracted {len(product_data) - 1} attributes.")

    except Exception as e:
        print(f"⚠️ Error: {e}")
        # If driver session died, reinitialize
        try:
            driver.quit()
        except:
            pass
        print("♻️ Restarting Chrome session...")
        driver = create_driver()
        continue  # skip current URL

    all_data.append(product_data)

    # Save progress every 10 products
    if idx % 10 == 0:
        pd.DataFrame(all_data).to_csv(output_file, index=False, encoding="utf-8-sig")
        print(f"💾 Progress saved ({idx} products)")

# ========== STEP 3: Save final output ==========
driver.quit()
pd.DataFrame(all_data).to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"\n✅ Extraction completed. File saved: {output_file}")


🔗 Total product links found: 2862

[1/2862] 🔍 Extracting: https://www.lenskart.com/lenskart-hustlr-lb-e14058-c23-eyeglasses.html
✅ Extracted 21 attributes.

[2/2862] 🔍 Extracting: https://www.lenskart.com/lenskart-hustlr-lb-e14058-H-c39-eyeglasses.html
✅ Extracted 21 attributes.

[3/2862] 🔍 Extracting: https://www.lenskart.com/lenskart-hustlr-lb-e14058-xw-c54-eyeglasses.html
✅ Extracted 21 attributes.

[4/2862] 🔍 Extracting: https://www.lenskart.com/vincent-chase-vc-e13526-c1-eyeglasses.html
✅ Extracted 22 attributes.

[5/2862] 🔍 Extracting: https://www.lenskart.com/lenskart-blu-lb-e18341-silver -eyeglasses.html
✅ Extracted 16 attributes.

[6/2862] 🔍 Extracting: https://www.lenskart.com/vincent-chase-vc-e13526-c3-eyeglasses.html
✅ Extracted 23 attributes.

[7/2862] 🔍 Extracting: https://www.lenskart.com/lenskart-hustlr-lb-e14058-c34-eyeglasses.html
✅ Extracted 21 attributes.

[8/2862] 🔍 Extracting: https://www.lenskart.com/lenskart-blu-lb-e13737-c2-eyeglasses.html
✅ Extracted 22 attrib

KeyboardInterrupt: 